# Using the `cinterp` Module

In [1]:
import os
import sys
import glob
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Local modules
import nexrad
import cinterp

# iRadar
sys.path.append(os.path.expanduser('~/Developer/iradar/python'))
import iradar

# Read a NEXRAD Sweep

In [3]:
files = glob.glob(os.path.expanduser('~/Downloads/88d/KDDC/*V06'))
filename = files[0]

azimuth_count = 360
range_count = 800

sweeps = nexrad.read(filename)

z = sweeps['z'][0, :, :range_count]

az = sweeps['aa'][0, :]
rr = 1.0e-3 * sweeps['rr'][:range_count]

roll_amount = int(az[0] * 180.0 / np.pi)
z_rolled = np.roll(z, roll_amount, axis=0)

In [4]:
sweeps

{'longitude': -99.96888732910156,
 'latitude': 37.760833740234375,
 'time': time.struct_time(tm_year=2020, tm_mon=1, tm_mday=29, tm_hour=1, tm_min=17, tm_sec=29, tm_wday=2, tm_yday=29, tm_isdst=0),
 'ee': array([0.48339844, 0.87890625, 1.3183594 ], dtype=float32),
 'aa': array([[1.05015366, 1.06784238, 1.08538728, ..., 0.99881324, 1.0161664 ,
         1.0337113 ],
        [1.53930178, 1.55689463, 1.57429572, ..., 1.48719437, 1.5050269 ,
         1.52223625],
        [2.09848572, 2.115743  , 2.13295235, ..., 2.045851  , 2.06358765,
         2.080797  ]]),
 'rr': array([  2125.,   2375.,   2625., ..., 459375., 459625., 459875.],
       dtype=float32),
 'z': array([[[ -8. ,  22.5,  22. , ..., -32.5, -32.5, -32.5],
         [ 46. ,  25.5,  16.5, ..., -32.5, -32.5, -32.5],
         [ 44.5,  -3. ,  13. , ..., -32.5, -32.5, -32.5],
         ...,
         [-33. , -33. ,  14. , ..., -32.5, -32.5, -32.5],
         [-13. ,  40. ,  19. , ..., -32.5, -32.5, -32.5],
         [-13.5, -10. ,  -8. , ..

In [2]:
%matplotlib inline
plt.rcParams['axes.facecolor'] = 'black'
zmap = iradar.colormap.zmap()
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', zmap[:, :3], N=len(zmap))

NameError: name 'iradar' is not defined

# Cell Indices

In [3]:
cells = cinterp.data.kddc_cells2

NameError: name 'cinterp' is not defined

# Calling `cinterp` of the CInterp Module

In [ ]:
# The smaller region that contains all the indices of the cells
ia_sub, ir_sub = cinterp.cluster_ar_indices(cells)

print('ia_sub = {}   ir_sub = {}'.format(ia_sub, ir_sub))

# One would expect +3 at the end to be sufficient but pcolor has this weird behavior
ia_sub = np.arange(ia_sub[0] - 2, ia_sub[1] + 4)
ir_sub = np.arange(ir_sub[0] - 2, ir_sub[1] + 4)

In [ ]:
z_interp = cinterp.cinterp(z, az, rr, cells)

In [ ]:
plt.figure(dpi=200)
plt.pcolormesh(ir_sub, ia_sub, z_rolled[ia_sub, :][:, ir_sub], cmap=cmap)
plt.plot(cells[:, 1] + 0.5, cells[:, 0] + 0.5, 'xw', markersize=4)
plt.clim((-32, 96))
plt.colorbar()
plt.grid()

plt.figure(dpi=200)
plt.pcolormesh(rr[ir_sub], ia_sub, z_interp[ia_sub, :][:, ir_sub], cmap=cmap)
plt.clim((-32, 96))
plt.colorbar()
plt.grid()